In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
#!unrar x "/content/drive/MyDrive/train_conv.rar" "/content/"
#!unrar x "/content/drive/MyDrive/val_conv.rar" "/content/"

In [3]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import cv2

In [4]:
from keras.applications import VGG16


In [5]:
df=pd.read_csv('df2.csv')


In [6]:
#df['image']=df['image'].apply(lambda x: x if x[-4:]=='.png' else np.nan)

In [7]:
#df=df.dropna().reset_index(drop=True)

In [8]:
df.dir='/'+df.type+'/'+df.image

In [9]:
df

,image,dir,cls,type
0,crop1_DJI_0002_1.jpg,/val/crop1_DJI_0002_1.jpg,1,val
1,crop1_DJI_0002_15.jpg,/val/crop1_DJI_0002_15.jpg,1,val
2,crop1_DJI_0002_23.jpg,/val/crop1_DJI_0002_23.jpg,1,val
3,crop1_DJI_0481_1.jpg,/val/crop1_DJI_0481_1.jpg,1,val
4,crop1_DJI_0481_101.jpg,/val/crop1_DJI_0481_101.jpg,1,val
...,...,...,...,...
33639,DJI_0121_975.jpg,/train/DJI_0121_975.jpg,99,train
33640,DJI_0121_98.jpg,/train/DJI_0121_98.jpg,99,train
33641,DJI_0121_982.jpg,/train/DJI_0121_982.jpg,99,train
33642,DJI_0121_990.jpg,/train/DJI_0121_990.jpg,99,train


In [10]:
size=576

In [11]:
train = ImageDataGenerator(rescale=1/255)
val = ImageDataGenerator(rescale=1/255)

In [12]:
train_generator = train.flow_from_directory(
    directory=r"/content/train_conv/",
    target_size=(size, size),
    color_mode="rgb",
    batch_size=16,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

Found 3013 images belonging to 102 classes.


In [13]:
valid_generator = train.flow_from_directory(
    directory=r"/content/val_conv/",
    target_size=(size, size),
    color_mode="rgb",
    batch_size=16,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

Found 1859 images belonging to 102 classes.


In [14]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('/content/mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')



In [15]:
model=tf.keras.applications.resnet50.ResNet50(include_top=True,
                                              input_shape=(size, size, 3),
                                              pooling=None,
                                              weights=None,#'imagenet',
                                              classes=102)
#model.add(layers.Dense(102))

In [16]:
'''model = models.Sequential()
#model.add(conv_base)
model.add(layers.Conv2D(256, kernel_size=(3,3),activation='relu',input_shape=(size, size, 3)))
#model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(size, size, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
#model.add(layers.Dense(256, activation='relu'))
model.add(layers.Flatten())

model.add(layers.Dense(102))'''

"model = models.Sequential()\n#model.add(conv_base)\nmodel.add(layers.Conv2D(256, kernel_size=(3,3),activation='relu',input_shape=(size, size, 3)))\n#model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(size, size, 3)))\nmodel.add(layers.MaxPooling2D((2, 2)))\nmodel.add(layers.Conv2D(128, (3, 3), activation='relu'))\nmodel.add(layers.MaxPooling2D((2, 2)))\nmodel.add(layers.Conv2D(128, (3, 3), activation='relu'))\n#model.add(layers.Dense(256, activation='relu'))\nmodel.add(layers.Flatten())\n\nmodel.add(layers.Dense(102))"

In [ ]:
model.summary()

In [18]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt,
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

In [19]:
history = model.fit(train_generator ,
                    epochs=15, 
                    validation_data=valid_generator,
                    callbacks=[earlyStopping, mcp_save, reduce_lr_loss])

Epoch 1/15
189/189 [==============================] - 280s 1s/step - loss: 3.8141 - accuracy: 0.1484 - val_loss: 6.2160 - val_accuracy: 0.0226 - lr: 0.0010
Epoch 2/15
189/189 [==============================] - 263s 1s/step - loss: 2.7823 - accuracy: 0.2612 - val_loss: 4.4122 - val_accuracy: 0.1135 - lr: 0.0010
Epoch 3/15
189/189 [==============================] - 262s 1s/step - loss: 2.3632 - accuracy: 0.3568 - val_loss: 33.2534 - val_accuracy: 0.1178 - lr: 0.0010
Epoch 4/15
189/189 [==============================] - 262s 1s/step - loss: 2.1121 - accuracy: 0.4006 - val_loss: 9.3133 - val_accuracy: 0.0807 - lr: 0.0010
Epoch 5/15
189/189 [==============================] - 262s 1s/step - loss: 1.8682 - accuracy: 0.4663 - val_loss: 4.6156 - val_accuracy: 0.3147 - lr: 0.0010
Epoch 6/15
189/189 [==============================] - 261s 1s/step - loss: 1.6242 - accuracy: 0.5224 - val_loss: 13.6077 - val_accuracy: 0.1259 - lr: 0.0010
Epoch 7/15
189/189 [==============================] - 263s 1s/

In [20]:
'''from keras import backend as K 

#model = load_model('model.h5')

inp = model.input                                         # input placeholder
out = [layer.output for layer in model.layers]            # all layer outputs
get_outputs = K.function([inp, K.learning_phase()], out)   

print(get_outputs)'''


"from keras import backend as K \n\n#model = load_model('model.h5')\n\ninp = model.input                                         # input placeholder\nout = [layer.output for layer in model.layers]            # all layer outputs\nget_outputs = K.function([inp, K.learning_phase()], out)   \n\nprint(get_outputs)"

In [21]:
from keras import backend as K

layer_dict = dict([(layer.name, layer) for layer in model.layers])

# input tensor
input_tensor = model.input

# output tensor of the given layer
layer_output = layer_dict['conv2d_1'].output

# get the output with respect to the input
func = K.function([input_tensor], [layer_output])

KeyError: ignored

In [ ]:
'''layer_outs = func([img, 1.])                                                
print(layer_outs)'''


In [ ]:
#preds=model.predict(valid_generator)